In [13]:
import time
import warnings
import requests
import pandas as pd
from IPython.display import clear_output

warnings.filterwarnings("ignore")

In [14]:
def import_data_to_redcap(api_token,path_to_csv):
    '''
    api_token: token used for secure authentifocation in redcap database,
    path_to_csv: csv_file stored data that in correct csv format can be imported to oue defined redcap database
    '''
    
    myFile = open(path_to_csv, encoding='utf-8')
    total_records = pd.read_csv(path_to_csv).shape[0]
    text = myFile.readline()
    header = text
    insert_content_str = header
    # chunk size is the number of records to import in one http post request
    chunk_size = 10000
    row_index = 0
    total_time = 0
    while text != "":
        text = myFile.readline()
        insert_content_str += text
        row_index +=1
        if(row_index%chunk_size==0):
            start_time = time.time()
            data = {
                    'token': api_token,
                    'content': 'record',
                    'action': 'import',
                    'format': 'csv',
                    'type': 'flat',
                    'overwriteBehavior': 'normal',
                    'forceAutoNumber': 'false',
                    'data': insert_content_str,
                    'returnContent': 'count',
                    'returnFormat': 'json'
                    }
            r = requests.post('https://redcap.wehi.edu.au/api/',data=data)
            end_time = time.time()
            total_time += end_time-start_time
         
            if(str(r.status_code) != '200'):
                print("Error occur at row {}. Response status code: {}. Response content: {}".format(row_index, r.status_code, r.content.decode('utf-8')))
            else:
                clear_output(wait = True)
                print("Already {}/{} records upload successful".format(row_index,total_records))
            insert_content_str = header
        
    

    # dont forget the last part of records
    start_time = time.time()
    data = {
                    'token': '27141BE34CC0C727A7E39F370E5CD701',
                    'content': 'record',
                    'action': 'import',
                    'format': 'csv',
                    'type': 'flat',
                    'overwriteBehavior': 'normal',
                    'forceAutoNumber': 'false',
                    'data': insert_content_str,
                    'returnContent': 'count',
                    'returnFormat': 'json'
                    }
    r = requests.post('https://redcap.wehi.edu.au/api/',data=data)
    end_time = time.time()
    total_time += end_time-start_time
    if r.status_code != 200:
        print("Error occur at row {}. Response status code: {}. Response content: {}".format(row_index, r.status_code, r.content.decode('utf-8')))
    else:
        clear_output(wait = True)
        print("{}/{} records have been uploaded successfully!".format(row_index-1,total_records))


    myFile.close()
    print("{} records have been uploaded to the REDCap database! Time taken: {} hours".format(row_index-1,round(total_time/3600,3)))  
    
    return 

In [16]:
# Change the API token & path_to_csv to your own
api_token = '27141BE34CC0C727A7E39F370E5CD701'
path_to_csv = 'sep/av_patient.csv'
import_data_to_redcap(api_token,path_to_csv)

100/100 records uploaded successfully
100 records have been uploaded to the redcap database. Time taken: 0.0 hours
